In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("time_series_data.csv")

# Convert DrawDate column to datetime format
df["DrawDate"] = pd.to_datetime(df["DrawDate"], format="%Y%m%d")

# Sort data by date
df.sort_values(by="DrawDate", inplace=True)

# Extract features and target variable
X = df["DrawDate"]
y = df[["DrawnNo1", "DrawnNo2", "DrawnNo3", "DrawnNo4", "DrawnNo5", "DrawnNo6"]]

# Split data into train and test sets
split_index = int(len(df) * 0.8)
train_data = y.iloc[:split_index]
test_data = y.iloc[split_index:]

# Scale the data using MinMaxScaler
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)


# Define function to create LSTM model
def create_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(LSTM(64, input_shape=(None, n_inputs)))
    model.add(Dense(n_outputs, activation="linear"))
    return model


# Define function to prepare data for LSTM model
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i : i + n_steps])
        y.append(data[i + n_steps])
    X = np.array(X)
    y = np.array(y)
    return X, y


# Define parameters for LSTM model
n_steps = 3
n_inputs = 6
n_outputs = 6

# Prepare data for LSTM model
X_train, y_train = prepare_data(train_data_scaled, n_steps)
X_test, y_test = prepare_data(test_data_scaled, n_steps)

# Reshape input data for LSTM model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_inputs))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_inputs))

# Create LSTM model
model = create_model(n_inputs, n_outputs)

# Compile model
model.compile(optimizer="adam", loss="mse")

# Fit the model to the training data
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# Predict the next 5 numbers for each date in the test data
predicted_data_scaled = model.predict(X_test)
predicted_data = scaler.inverse_transform(predicted_data_scaled)

# Convert predicted data to dataframe
predicted_df = pd.DataFrame(predicted_data, columns=y.columns)

# Get the last date in the train data
last_date = df.iloc[split_index - 1]["DrawDate"]

# Generate dates for the next 5 days
next_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=5)

# Print the predicted numbers for the next 5 dates
print("Predicted numbers for the next 5 dates:")
for i, date in enumerate(next_dates):
    print("Date: ", date.strftime("%Y%m%d"))
    print("Numbers: ", predicted_df.iloc[i].values)
    print()




ModuleNotFoundError: No module named 'tensorflow'